# MTAOS add aberrations to M1M3+M2+hexapod #

This notebook is used for the level 3 integration tests from test plan LVV-P81 (https://jira.lsstcorp.org/secure/Tests.jspa#/testPlan/LVV-P81) as part of test cylce LVV-C176 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCycle/LVV-C176). The following tests are currently run as part of this notebook:

 - LVV-T2190 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2190)
 
Execution steps are separated by horizontal lines. Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA. 

Last executed by E. Dennihy 20210928

***

Load all the needed libraries.
Get the remotes ready
Code in the notebook including section: "Check the summary state of each CSC".

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
import os
import sys
import asyncio
import logging

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

***

Ready M1M3:
Raise mirror, turn on FB, clear forces

Need to have M1M3 LUT use its inclinometer.

***

Ready M2:
Turn on FB, clear forces

Need to have M2 LUT use its inclinometer

***

Get camera hexapod ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position


***

Get M2 hexapod ready: check config; make sure LUT is on, and has valid inputs; make sure hex is at LUT position

***

add 1um of z7 to the system via OFC

Compare the corrections sent vs forces and position changes applied. This is currently done in a separate notebook.

In [ ]:
wavefront_errors = np.zeros(19)

In [ ]:
wavefront_errors[3] += 0.1 # add1 um to z7

In [ ]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

This command primes the corrections, the issueCorrection command is needed to actually command them to be sent

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

***
Make plots using telemetry from each component to verify the changes in the DOFs. This step does not currently involve running any commands in this notebook. This step must be verified using a separate noteboook. 

***
reset the corrections using the resetCorrection command

Compare the corrections sent vs forces and position changes applied (these are all expected to be zero). This is currently done in a separate notebook or on Chronograf. 

In [ ]:
await mtcs.rem.mtaos.cmd_resetCorrection.start()

***
add 2um of z7 to the system via OFC

Compare the corrections sent vs forces and position changes applied. This is currently done in a separate notebook or on Chronograf. 

In [ ]:
wavefront_errors[3] = 1.0 # add 2.0 um of z7

In [ ]:
await mtcs.rem.mtaos.cmd_addAberration.set_start(wf=wavefront_errors, timeout=10)

In [ ]:
await mtcs.rem.mtaos.cmd_issueCorrection.start(timeout=60.)

***
Check that the corrections in step 10 are twice of those in step 7. This step does not currently involve running any commands in this notebook. This step must be verified using a separate noteboook. 

***
Wrap up. Put each component to the following states:
mtaos --> standby
m1m3 --> lower mirror --> standby
m2 --> standby
camera hex --> standby
m2 hex --> standby

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
await mtcs.standby()